In [22]:
import argparse
import os
import ray
from torch import nn

from typing import *
from ray import air
from ray import tune
from baselines.train.configs import get_experiment_config
from ray.rllib.algorithms import ppo
from ray.tune import registry
from ray.air.integrations.wandb import WandbLoggerCallback
from baselines.train import make_envs

In [2]:
ckp = "results/torch/pd_matrix/PPO_meltingpot_397b4_00000_0_2024-05-21_13-58-39/checkpoint_007270"

registry.register_env("meltingpot", make_envs.env_creator)


In [3]:
# del trainer
trainer:ppo.PPO = ppo.PPO.from_checkpoint(ckp)
trainer.restore(ckp)
print(f"Continuing training from {ckp}")

2024-05-24 07:04:37,880	WARNING algorithm_config.py:2534 -- Setting `exploration_config={}` because you set `_enable_rl_module_api=True`. When RLModule API are enabled, exploration_config can not be set. If you want to implement custom exploration behaviour, please modify the `forward_exploration` method of the RLModule at hand. On configs that have a default exploration config, this must be done with `config.exploration_config={}`.
2024-05-24 07:04:37,882	WARNING algorithm_config.py:2548 -- Setting `exploration_config={'type': 'StochasticSampling'}` because you set `_enable_rl_module_api=False`. This exploration config was restored from a prior exploration config that was overriden when setting `_enable_rl_module_api=True`. This occurs because when RLModule API are enabled, exploration_config can not be set.
2024-05-24 07:04:37,885	WARNING algorithm_config.py:656 -- Cannot create PPOConfig from given `config_dict`! Property __stdout_file__ not supported.
2024-05-24 07:04:37,898	WARNIN

Continuing training from results/torch/pd_matrix/PPO_meltingpot_397b4_00000_0_2024-05-21_13-58-39/checkpoint_007270


In [4]:
print(trainer)
print(dir(trainer))
print(trainer.__dict__)

PPO
['__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__ray_actor_class__', '__ray_ready__', '__ray_terminate__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_allow_unknown_configs', '_allow_unknown_subkeys', '_annotated', '_automatic_evaluation_duration_fn', '_before_evaluate', '_checkpoint_cls', '_checkpoint_info_to_algorithm_state', '_close_logfiles', '_compile_iteration_results', '_counters', '_create_checkpoint_dir', '_create_local_replay_buffer_if_necessary', '_create_logger', '_env_id', '_episode_history', '_episodes_to_be_collected', '_episodes_total', '_evaluate_async', '_export_model', '_get_env_id_and_creator', '_get_latest_available_checkpoint', '_get_latest_local_available_c

In [35]:
# trainer.training_iteration()
worker:ray.rllib.evaluation.rollout_worker.RolloutWorker= trainer.workers.local_worker()
policy_spec = worker.policy_dict['agent_0']
policy_instance = worker.policy_map['agent_0']
nn_model:nn.Module = policy_instance.model 
print(dir(worker))
# print(isinstance(nn_model,nn.Module))
print("Total_params: ", sum([param.nelement() for param in nn_model.parameters()]))
# print(nn_model.state_dict().keys())
print(nn_model._modules.keys())
print(nn_model)
# print("Total_params: ", nn_model.total_parameters())

['__abstractmethods__', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_annotated', '_build_policy_map', '_call_callbacks_on_create_policy', '_ds_shards', '_get_complete_policy_specs_dict', '_get_input_creator_from_config', '_get_make_sub_env_fn', '_get_output_creator_from_config', '_lock', '_original_kwargs', '_update_filter_dict', '_update_policy_dict_with_marl_module', '_update_policy_map', 'add_policy', 'apply', 'apply_gradients', 'assert_healthy', 'async_env', 'callbacks', 'compute_gradients', 'config', 'creation_args', 'default_policy_class', 'env', 'env_context', 'env_creator', 'filters', 'find_free_port', 'for_policy', 'foreach_env', 'foreach_env_with_context', 